1. **Goal 1**:Train a fully supervised baseline neural network. For this goal, you will use a simple ResNet (provided below) and the DataLoader class you have been implementing to train a neural network to predict what plant species is present in a given image in a fully supervised way.
> * As a reminder, [fully supervised learning](https://towardsdatascience.com/supervised-vs-unsupervised-learning-14f68e32ea8d) involves training a model to predict the output `Y` from input `X`. For your project, `X` is an image of a plant

> >>![plant image](https://inaturalist-open-data.s3.amazonaws.com/photos/126743755/medium.jpg)

> * While `Y` is the name of the corresponding plant (i.e.: *Hesperis matronalis* or Dame's Rocket as it's more commonly known).

2. **Goal 2**: Train a simple triplet loss contrastive self-supervised baseline. This form of training has two training steps, unlike the classic fully supervised version.
> In the first step for self-supervised learning, we throw out the label `Y` for the first part of training, instead opting to use three images from our dataset, `X`, `f(X)`, and `~X`.
> In this case, `X` is an image, like above, `f(X)` is some form of *augmentation* of the image, like a flip or a crop

>> ![da](https://editor.analyticsvidhya.com/uploads/84333flip.jpg)
>
> Finally, `~X` is typically another image randomly sampled from the dataset, like:
> >>![plant image](https://inaturalist-open-data.s3.amazonaws.com/photos/25176952/medium.jpeg)

> In the second step of self-supervised learning, we take our trained neural network and *freeze the weights* (meaning, those real-valued parameters will no longer be updated by gradient descent during training). We then swap the last layer with a new one that corresponds to our classes from `Y` (in this case, that correspond to all the plant species we're trying identify). We then train our neural network for an additional amount of time, only updating this new output layer (this is called **feature extraction** which is explained along with other forms of transfer learning [here](https://stats.stackexchange.com/questions/255364/fine-tuning-vs-joint-training-vs-feature-extraction)).




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "gdrive/Shareddrives/197/project_1_dataset.zip" -d "/content"

In [ ]:
import pandas as pd
label_dict = {}
f_train = pd.read_csv('/content/project_1/train/observations.csv')
i = 0
for row in range(f_train.shape[0]):
  specie = f_train.iloc[row,4]
  if (specie not in label_dict):
    label_dict[specie] = i
    i += 1

In [ ]:
#label_dict = {}
#f_train = pd.read_csv('/content/project_1/train/observations.csv')
#for row in f_train.iterrows():
#  specie = row['taxon_id'] # or row.taxon_id #  f_train.iloc[row,4]
#  if (specie not in label_dict):
#    label_dict[specie] = i
#    i += 1

In [ ]:
# machine learning packages
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

# plotting packages
import matplotlib.pyplot as plt

# miscellaneous packages
import json
import datetime
from google.colab import drive
from tqdm import tqdm, trange
from types import SimpleNamespace

In [ ]:
# Neural network implementation for ResNet 18
# for a visual walkthrough of this neural network, see: https://towardsdatascience.com/understanding-and-visualizing-resnets-442284831be8
# code from: https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py
class BasicBlock(nn.Module):
  # the basic block implements a basic residual block
    expansion: int = 1

    def __init__(
            self,
            inplanes: int,
            planes: int,
            stride: int = 1,
            downsample = None,
            groups: int = 1,
            base_width = 64,
            dilation = 1,
            norm_layer = None):
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)
      # the actual residual part!
        out += identity
        out = self.relu(out)

        return out

In [ ]:
# sets up the convolutional layers for us all nice
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )
def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
# actual ResNet class implementation
class ResNet18(nn.Module):
    def __init__(
            self,
            block, # what kind of block to use. Small ResNets only use basic blocks
            layers, # number of BasicBlocks to make in each layer of ResNet
            num_classes: int = 64, # number of species
            groups: int = 1):
        super().__init__()

        norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
       # magic numbers to match the matrix sizes for matrix multiplication
        self.inplanes = 64
        self.dilation = 1
        # for ResNet18 and BasicBlock, the group and base_width must always be these vals
        self.groups = 1
        self.base_width = 64
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # at some point, you may want to do the math so that your feature
        # extraction layer has > 512 features in it. In the past, I've found
        # 2048 to be a better final layer size.
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

      # initialize the model weights to be small real values drawn from a Z normal distribution
      # turns out starting all the parameter values at 0 can lead to either untrainable or unstable NNs
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        for m in self.modules():
            if isinstance(m, BasicBlock) and m.bn2.weight is not None:
                nn.init.constant_(m.bn2.weight, 0)

    # sets up all the gross math so the weight matrices line up correctly
    def _make_layer(
        self,
        block,
        planes: int,
        blocks: int,
        stride: int = 1,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [ ]:
# Dataset and dataloader
from torch.utils.data import DataLoader
# TODO: copy in your dataset and dataloader classes here
import torch
from torch.utils.data import Dataset
import pandas as pd
import torchvision
import torchvision.transforms as transforms
from PIL import Image

class WildFlowerDataset(Dataset):
    def __init__(self, data_frame, img_dir, transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(csv)
        self.img_labels = data_frame
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
      # crop to square 150x150 pixels and pad images that are less than that
        photo_id_train = self.img_labels.iloc[idx,13]
        folder_num_train = photo_id_train // 1000000

        img_path = self.img_dir + "/" + str(folder_num_train) + "/" + str(photo_id_train) + ".png"
        img_og = Image.open(img_path)
        #ensure image is png
        #background = Image.new('RGBA', img_og.size, (255, 255, 255))
        #alpha_composite = Image.alpha_composite(background, img_og)
        #alpha_composite.save(img_og, 'png', quality=100)
        tensor_img = transforms.ToTensor()(img_og)
        tensor_img = tensor_img[:3, : , : ]

        if tensor_img.shape[0] < 3:  # Check the number of channels
            tensor_img = tensor_img.repeat(3, 1, 1)

        label = label_dict[self.img_labels.iloc[idx, 4]]

        if self.transform:
            tensor_img = self.transform(tensor_img)
        if self.target_transform:
            label = self.target_transform(label)

        crop = transforms.CenterCrop((150,150))
        image = crop(tensor_img)
        return image, label

In [ ]:
# Train loop
def train(args, model, device, train_loader, optimizer, epoch, logger, count, SummaryWriter):
    writer = SummaryWriter


  # have to remember to put your model in training mode!
    model.train()
    # logs loss per-batch to the console
    # stats is a range object from zero to len(train_loader)
    stats = trange(len(train_loader))
    indexer = epoch * len(train_loader)
    for batch_idx, (data, target)  in zip(stats, train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # standard softmax cross-entropy loss
        # explainer here: https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        # save train loss to tqdm bar
        stats.set_description(f'epoch {epoch}')
        stats.set_postfix(loss=loss.item())
        logger[batch_idx+indexer] = loss.item()

        #added for logging
        writer.add_scalar('Train/Loss', loss.item(), count)
        count+=1
    stats.close()
    return logger

In [ ]:
# Test loop
def test(model, device, test_loader, epoch, logger, count, SummaryWriter):
  #logging
    writer = SummaryWriter
  # need to put into eval() mode to ensure model outputs are deterministic
    model.eval()
    test_loss = 0
    correct = 0
    top5correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, total=len(test_loader), desc=f'testing epoch {epoch}'):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            #logging for test loss
            writer.add_scalar('Test/Loss', (F.cross_entropy(output, target, reduction='sum').item()), count)
            count+=1
            # see if the highest predicted class was the right class
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

            top5, indices = output.topk(5, dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()
            for c in range(indices.size(1)):
              c_tens = indices[:,c]
              top5correct += c_tens.eq(target.view_as(c_tens)).sum().item()
    # get average test loss
    test_loss /= len(test_loader.dataset)
    print(f'test set avg loss: {round(test_loss, 6)} Acc: {correct}/{len(test_loader.dataset)}: {100*round(correct / len(test_loader.dataset), 4)}%')
    logger['loss'][epoch] = round(test_loss, 6)
    logger['1accuracy'][epoch] = 100*round(correct / len(test_loader.dataset), 6)
    print(f'test set avg loss top-5: {round(test_loss, 6)} Acc: {top5correct}/{len(test_loader.dataset)}: {100*round(top5correct / len(test_loader.dataset), 4)}%')
    logger['5accuracy'][epoch] = 100*round(top5correct / len(test_loader.dataset), 6)
    return logger

In [ ]:
# set up hyperparameters
hyperparams = {
    'batch_size' : 64, # 64
    'test_batch_size' : 1000, #1000
    'epochs' : 12,
    'lr' : 0.001,
    'gamma' : 0.7,
    'seed' : 1,
}
hyperparams = SimpleNamespace(**hyperparams)

In [ ]:
# TODO: instantiate your dataset and loader here
import torchvision.transforms as transforms
f_test = pd.read_csv('/content/project_1/test/observations.csv')
f_val = pd.read_csv('/content/project_1/validation/observations.csv')
f_test = f_test.loc[f_test['taxon_id'].isin(label_dict)]
train_dset = WildFlowerDataset(f_train,'/content/project_1/train/images' , transform=None, target_transform=None)
test_dset = WildFlowerDataset(f_test,'/content/project_1/test/images' , transform=None, target_transform=None)
val_dset = WildFlowerDataset(f_val,'/content/project_1/validation/images' , transform=None, target_transform=None)
train_loader = DataLoader(train_dset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dset, batch_size=64, shuffle=True)

In [ ]:
# this loads in a randomly initialized ResNet18
# if you want to try using imagenet-pretrained weights at some point,
# check out this article: https://pytorch.org/vision/master/models.html
model = ResNet18(BasicBlock, [2, 2, 2, 2])

# set up optimizer
optimizer = optim.Adam(model.parameters(), lr=hyperparams.lr)

# set up cuda devices
device = torch.device("cuda")
torch.manual_seed(hyperparams.seed);
model = model.to(device)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
# Running main code loop
train_log, test_log = {}, {}
test_log['loss'] = {}
test_log['1accuracy'] = {}
test_log['5accuracy'] = {}
#for train logging
countTrainBatch = 0
countTestBatch = 0
countEpoch = 0
writer = SummaryWriter("logs")
for epoch in range(hyperparams.epochs):
    print(f'starting epoch {epoch}')
    train_log = train(hyperparams, model, device, train_loader, optimizer, epoch, train_log, countTrainBatch, writer)
    countTrainBatch += len(train_loader)
    test_log = test(model, device, test_loader, epoch, test_log,countTestBatch,writer)
    writer.add_scalar('Top-1 Test/Accuracy', test_log['1accuracy'][epoch], countEpoch)
    writer.add_scalar('Top-5 Test/Accuracy', test_log['5accuracy'][epoch], countEpoch)
    countTestBatch += len(test_loader)
    countEpoch+=1


starting epoch 0


testing epoch 0: 100%|██████████| 921/921 [02:03<00:00,  7.46it/s]


test set avg loss: 2.363505 Acc: 20857/58894: 35.410000000000004%
test set avg loss top-5: 2.363505 Acc: 40161/58894: 68.19%
starting epoch 1


testing epoch 1: 100%|██████████| 921/921 [02:02<00:00,  7.49it/s]


test set avg loss: 2.112585 Acc: 23978/58894: 40.71%
test set avg loss top-5: 2.112585 Acc: 44955/58894: 76.33%
starting epoch 2


...

test set avg loss: 1.781952 Acc: 37806/58894: 64.19%
test set avg loss top-5: 1.781952 Acc: 52206/58894: 88.64%
starting epoch 10


testing epoch 10: 100%|██████████| 921/921 [02:02<00:00,  7.53it/s]


test set avg loss: 2.236243 Acc: 34316/58894: 58.269999999999996%
test set avg loss top-5: 2.236243 Acc: 49925/58894: 84.77%
starting epoch 11


testing epoch 11: 100%|██████████| 921/921 [02:02<00:00,  7.51it/s]

test set avg loss: 2.034964 Acc: 36274/58894: 61.59%
test set avg loss top-5: 2.034964 Acc: 51116/58894: 86.79%


In [ ]:
# Save hyperparameters and model weights

# You'll want to think about how to structure your
# data drive so it doesn't get too messy from checkpoints
# alternately, you can replace this with wandb if you go that route
checkpoint_path = '/content/gdrive/Shareddrives/197/project_1'
savename = datetime.datetime.now().strftime("%H%M_%m%d%Y")
# save your hyperparameters
with open(f"{checkpoint_path}hyperparams_{savename}.json", 'w') as f:
  json.dumps(vars(hyperparams))

# save your model state
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch' : hyperparams.epochs,
            'train_log' : train_log,
            'test_log' : test_log,

            }, f"{checkpoint_path}model_{savename}.tar")

In [ ]:
!pip install tensorflow

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs

In [ ]:
#Add train logging
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np

"""
writer = SummaryWriter("logs")

for n_iter in range(14):
    writer.add_scalar('Loss/train', train_log[n_iter], n_iter)
    writer.add_scalar('Loss/test', test_log['loss'][n_iter], n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', test_log['accuracy'][n_iter], n_iter)
"""

'\nwriter = SummaryWriter("logs")\n\nfor n_iter in range(14):\n    writer.add_scalar(\'Loss/train\', train_log[n_iter], n_iter)\n    writer.add_scalar(\'Loss/test\', test_log[\'loss\'][n_iter], n_iter)\n    writer.add_scalar(\'Accuracy/train\', np.random.random(), n_iter)\n    writer.add_scalar(\'Accuracy/test\', test_log[\'accuracy\'][n_iter], n_iter)\n'